# Import environment

In [ ]:
pip install --upgrade gensim

In [ ]:
pip install nltk

In [ ]:
pip install spacy

In [ ]:
pip install --user en_core_web_sm

In [214]:
pip install --user pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install wget

In [218]:
import numpy as np 
import pandas as pd
import spacy
from gensim import corpora, models, similarities
import gensim
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
import wget
warnings.filterwarnings('ignore', category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
url = "https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.0/en_core_web_sm-2.3.0.tar.gz"
wget.download(url)

100% [........................................................................] 12046667 / 12046667

'en_core_web_sm-2.3.0.tar.gz'

In [4]:
pip install --user en_core_web_sm-2.3.0.tar.gz 

Processing c:\users\admin\desktop\business project\en_core_web_sm-2.3.0.tar.gz
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.0-py3-none-any.whl size=12048612 sha256=86bf2b8683ece1404508f94260a096bb450d8828b9cb7cb90a1b0c4f6625e2f5
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\cc\1b\71\567d1ae653e24e68f88b9f05dbcbcbddf1812bed3163339946
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.3.0
    Uninstalling en-core-web-sm-2.3.0:
      Successfully uninstalled en-core-web-sm-2.3.0
Note: you may need to restart the kernel to use updated packages.


# Cleaning data

In [5]:
df = pd.read_csv('C:/Users/Admin/Desktop/Business Project/CRv2-DOCS.csv',header=None)

In [6]:
df.shape

(229, 3)

In [29]:
df.head()

,0,1,2
0,0,CRv2/Chunk1/BR_Cross-London_Rail_Link_Nov_1980...,CROSSLONDON RAIL LINK This booklet is about a...
1,1,CRv2/Chunk1/British_Railways_(London)_Bill_Lor...,British Railways London Bill Lords By Order O...
2,2,CRv2/Chunk1/Greater_London_Report_27_Nov_1974....,THE GREATER LONDON REPORT CONTENTS Railways s...
3,3,CRv2/Chunk1/House_of_Commons._Fifth_report_fro...,HOUSE OF COMMONS Fifth Report from the TRANSP...
4,4,CRv2/Chunk1/House_of_Commons._First_report_fro...,HOUSE OF COMMONS o Q Q First Report from the...


In [185]:
texts = df[2]

In [9]:
stopwords = stopwords.words("english")

In [10]:
def lemmatisation(text, allowed_postages=["NOUN","ADJ","VERB","ADV"]):
    nlp = spacy.load("en_core_web_sm",disable=["parser","ner"])
    text_out=[ ]
    doc = nlp(text)
    for token in doc:
        if token.pos_ in allowed_postages:
            text_out.append(token.lemma_)
    return (text_out)

In [189]:
de = [ ]
for x in range(0,len(texts)):
    if len(texts[x]) > 1000000:
        de.append(x)
for x in de:
    print(x,len(texts[x]),sep = ":")

3:1455266
52:1593514
58:2345574
73:1888094
74:3089219
98:1708577
109:1621136
111:1405510
166:11237569
184:1127302
186:1314130
190:1239113
196:1258279
215:1080873
219:1843436
222:1282861
223:1375401


# Some Documents Exceed the Maximum Capacity of spacy library, which is 10^6. Problem solved with two loop.

In [190]:
for x in de:
    if len(texts[x]) > 1000000:
        str_surplus = texts[x][1000000:len(texts[x])]
        texts = texts.append(pd.Series(str_surplus,index=[len(texts)]))
        texts[x]=texts[x][0:1000000]
        for i in range(0,len(texts)):
            if len(texts[i]) > 1000000:
                de.append(i)

In [193]:
%%time
lemma_list=[]
for text in texts:
    lemma=lemmatisation(text)
    lemma_list.append(lemma)

Wall time: 12min 40s


# Bag of Words

In [207]:
id2word = corpora.Dictionary(lemma_list)

corpus = []
for text in lemma_list:
    new = id2word.doc2bow(text)
    corpus.append(new)
    
print(corpus[0][0:20])

[(0, 1), (1, 1), (2, 3), (3, 1), (4, 1), (5, 3), (6, 2), (7, 7), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 7), (16, 2), (17, 1), (18, 2), (19, 1)]


# LDA modelling

In [212]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word = id2word,
                                           num_topics = 30,
                                           random_state = 42,
                                           update_every = 1,
                                           chunksize = 100,
                                           passes = 10,
                                           alpha = "auto")

Wall time: 58.8 s


# Visualisation

In [220]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.053423 -0.302117       1        1  13.964214
29    -0.072493 -0.357130       2        1  11.300998
19    -0.107186 -0.323719       3        1  10.867098
18    -0.025149 -0.306412       4        1   9.641561
26     0.172725 -0.332557       5        1   8.979317
12     0.114743 -0.305499       6        1   7.516408
17     0.170527 -0.295912       7        1   7.326611
7      0.065680 -0.311182       8        1   6.116485
9      0.012663 -0.361626       9        1   5.518038
8     -0.210247 -0.303385      10        1   5.470711
6      0.024502 -0.313628      11        1   4.609479
10    -0.181015 -0.280633      12        1   3.817656
3      0.065164 -0.358702      13        1   2.924424
5     -0.362835 -0.162687      14        1   1.945280
28     0.022585  0.269775      15        1   0.000535
1      0.023406  0.269664      16        1   0.000377
11     0.024307  0.269623      17        1   0.000061
25     0.024307  0.269637      18        1   0.000061
2      0.024308  0.269642      19        1   0.000060
27     0.024311  0.269688      20        1   0.000059
14     0.024311  0.269689      21        1   0.000059
16     0.024311  0.269694      22        1   0.000059
13     0.024309  0.269687      23        1   0.000058
4      0.024312  0.269709      24        1   0.000058
24     0.024312  0.269715      25        1   0.000057
23     0.024313  0.269726      26        1   0.000057
22     0.024313  0.269726      27        1   0.000057
15     0.024312  0.269722      28        1   0.000056
21     0.024314  0.269741      29        1   0.000055
20     0.024314  0.269751      30        1   0.000052, topic_info=           Term          Freq         Total Category  logprob  loglift
667   transport  38109.000000  38109.000000  Default  30.0000  30.0000
1518      local  18160.000000  18160.000000  Default  29.0000  29.0000
713        work  22856.000000  22856.000000  Default  28.0000  28.0000
519        rail  28771.000000  28771.000000  Default  27.0000  27.0000
647       think  18016.000000  18016.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
579         see      0.000031  13897.533592  Topic30 -11.2774  -5.4407
662     traffic      0.000031  10180.628821  Topic30 -11.2774  -5.1296
69          can      0.000031  30935.688515  Topic30 -11.2774  -6.2410
306     improve      0.000031   8602.390887  Topic30 -11.2775  -4.9612
433    operator      0.000031   8783.423315  Topic30 -11.2776  -4.9821

[2900 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
7097       8  0.991621    BALDWIN
18664      9  0.981623     BEFORE
2604       2  0.020346  COMMITTEE
2604       3  0.253397  COMMITTEE
2604       4  0.088782  COMMITTEE
...      ...       ...        ...
721       14  0.002277       year
27505      9  0.916459      yeson
29831      4  0.058092  youngster
29831      5  0.929475  youngster
8616       8  0.997616     zoning

[5625 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 30, 20, 19, 27, 13, 18, 8, 10, 9, 7, 11, 4, 6, 29, 2, 12, 26, 3, 28, 15, 17, 14, 5, 25, 24, 23, 16, 22, 21])